In [1]:
from copy import copy
from tqdm import tqdm
import pandas as pd
import math

In [2]:
data = pd.read_excel("../data/Calculations.xlsx", header=0)
data.head(2)

,Valve,GPM,Unnamed: 2,Network,Unnamed: 4,Unnamed: 5,Controler Valves No,75,Pump Unit Estimation GPM,1500,Unnamed: 10,Unnamed: 11
0,H-B62,36.0,NaN,B,4328.0,SUM B,No Of controlers,7.373333,NaN,NaN,NaN,NaN
1,J-B48,11.0,NaN,B,4053.0,SUM D,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
controller_valves_capacity = 75
pump_unit_estimated_gpm = 200

TWIN_PUMP = 2
TRIPLET_PUMP = 3

twin_pump_decimals = [ 0, 1, 2, 4, 5, 8, 9 ]
triplet_pump_decimals = [ 3, 7 ] 



In [4]:
data['gpm_int'] = data['GPM'].apply(lambda x: math.ceil(x))

data['valve_type_key'] = data.Valve.astype(str).apply(lambda x: x.strip().split('-')[-1][0])

In [5]:
valve_type_keys = data.valve_type_key.unique().tolist()
valve_type_keys

['B', 'D', 'S']

In [6]:
b_valves = data.loc[data['valve_type_key'] == 'B'][['Valve', 'gpm_int']]   
b_valves

,Valve,gpm_int
0,H-B62,36
1,J-B48,11
418,H-B60,43
419,K-B80,28
420,K-B81,23
...,...,...
548,A-B05,24
549,A-B04,33
550,A-B03,19
551,A-B02,36


In [7]:
def knapsack(weights, values, capacity, max_items):
    n = len(weights)
    # Create a table to store the maximum values at each capacity and number of items
    dp = [[[0 for _ in range(capacity + 1)] for _ in range(max_items + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, max_items + 1):
            for w in range(1, capacity + 1):
                if weights[i - 1][0] <= w:
                    dp[i][j][w] = max(values[i - 1] + dp[i - 1][j - 1][w - weights[i - 1][0]], dp[i - 1][j][w])
                else:
                    dp[i][j][w] = dp[i - 1][j][w]

    # Find the selected items
    selected_items = []
    w = capacity
    for i in range(n, 0, -1):
        if dp[i][max_items][w] != dp[i - 1][max_items][w]:
            selected_items.append(weights[i-1])
            w -= weights[i - 1][0]
            max_items -= 1

    return selected_items

In [14]:
def get_groups(valve_data):

    total_valves_gpm = valve_data.gpm_int.sum()
    total_num_valves = len(valve_data)
    num_batches = round(total_valves_gpm/pump_unit_estimated_gpm, 1)

    pump_type =  TWIN_PUMP if (num_batches - int(num_batches))*10 in twin_pump_decimals else TRIPLET_PUMP
    per_pump_gpm = pump_unit_estimated_gpm/pump_type
    print(per_pump_gpm)

    num_controllers_needed = math.ceil(len(valve_data)/controller_valves_capacity)
    num_batches = math.ceil(num_batches)
    batch_capacity = math.ceil( len(valve_data) / num_batches )

    values = [ 1 for _ in range(len(valve_data))]
    weights = list(zip(valve_data['gpm_int'].tolist(), valve_data['Valve'].tolist()))
    weights.sort(key=lambda x: x[0])


    all_groups = []
    for run_ in tqdm(range(num_batches)):

        group_ = knapsack(weights, values, pump_unit_estimated_gpm, batch_capacity)
        all_groups.append(copy(group_))
        for element in group_:
            weights.remove(element)
    
    inverted_groups = []
    for idx, group_ in enumerate(all_groups):

        group_ = [ ( x[-1], data.loc[data['Valve'] == x[-1]]['GPM'].item() ) for x in group_ ]

        names = [ x[0] for x in group_]
        values = [ x[1] for x in group_]

        group_total_gpm = sum(values)
        number_of_pumps_required = math.ceil(group_total_gpm/per_pump_gpm)

        inverted_groups.append((f"Group ID: {idx}", "", f"Group Total GPM: {group_total_gpm}", "", f"Pump Works: {number_of_pumps_required}/{pump_type}"))
        inverted_groups.append(names)
        inverted_groups.append(values)

    return pd.DataFrame(inverted_groups).fillna("")


In [16]:
for key in valve_type_keys:

    per_key_valves = data.loc[data['valve_type_key'] == key][['Valve', 'gpm_int']]  
    df= get_groups(b_valves)
    break
df

66.66666666666667


100%|██████████| 22/22 [00:00<00:00, 41.74it/s]


,0,1,2,3,4,5,6
0,Group ID: 0,,Group Total GPM: 69.0,,Pump Works: 2/3,,
1,G-B119,J-B48,J-B49,K-B73,B-B36,E-B123,H-B57
2,12.0,11.0,10.0,10.0,9.0,9.0,8.0
3,Group ID: 1,,Group Total GPM: 90.0,,Pump Works: 2/3,,
4,I-B47,A-B16,B-B23,G-B120,G-B117,A-B15,I-B111
...,...,...,...,...,...,...,...
61,G-B114,B-B24,F-B103,,,,
62,54.0,53.0,53.0,,,,
63,Group ID: 21,,Group Total GPM: 173.0,,Pump Works: 3/3,,
64,I-B130,E-B136,H-B116,,,,


In [ ]:
#uploaded_file = st.file_uploader("Choose a file")
#controller_valves_capacity = st.number_input('Controller Valves Capacity')
#pump_unit_estimated_gpm = st.number_input('Pump Unit Estimated GPM')
'''
if uploaded_file is not None and controller_valves_capacity is not None and pump_unit_estimated_gpm is not None:
    data=pd.read_excel(uploaded_file)

    #controller_valves_capacity = 75
    #pump_unit_estimated_gpm = 200

    TWIN_PUMP = 2
    TRIPLET_PUMP = 3

    twin_pump_decimals = [ 0, 1, 2, 4, 5, 8, 9 ]
    triplet_pump_decimals = [ 3, 7 ] 

    data['gpm_int'] = data['GPM'].apply(lambda x: math.ceil(x))

    data['valve_type_key'] = data.Valve.astype(str).apply(lambda x: x.strip().split('-')[-1][0])


    valve_type_keys = data.valve_type_key.unique().tolist()


    for key in valve_type_keys:



        per_key_valves = data.loc[data['valve_type_key'] == key][['Valve', 'gpm_int', 'GPM']]  
        st.subheader(f'Valves Group {key}, total GPM = {per_key_valves["GPM"].sum()}', divider='rainbow')
        df= get_groups(per_key_valves)

        st.dataframe(df, use_container_width=True)
        st.divider()

'''


### NEW

In [19]:
def knapsack(weights, values, capacity, max_items):
    n = len(weights)
    # Create a table to store the maximum values at each capacity and number of items
    capacity = math.ceil(capacity)
    max_items = math.ceil(max_items)
    dp = [[[0 for _ in range(capacity + 1)] for _ in range(max_items + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, max_items + 1):
            for w in range(1, capacity + 1):
                if weights[i - 1][0] <= w:
                    dp[i][j][w] = max(values[i - 1] + dp[i - 1][j - 1][w - weights[i - 1][0]], dp[i - 1][j][w])
                else:
                    dp[i][j][w] = dp[i - 1][j][w]

    # Find the selected items
    selected_items = []
    w = capacity
    for i in range(n, 0, -1):
        if dp[i][max_items][w] != dp[i - 1][max_items][w]:
            selected_items.append(weights[i-1])
            w -= weights[i - 1][0]
            max_items -= 1

    return selected_items

def get_groups(valve_data):

    total_valves_gpm = valve_data.gpm_int.sum()
    total_num_valves = len(valve_data)
    num_batches = round(total_valves_gpm/pump_unit_estimated_gpm, 1)

    pump_type =  TWIN_PUMP if (num_batches - int(num_batches))*10 in twin_pump_decimals else TRIPLET_PUMP
    per_pump_gpm = pump_unit_estimated_gpm/pump_type

    #num_controllers_needed = math.ceil(len(valve_data)/controller_valves_capacity)
    num_batches = math.ceil(num_batches)
    batch_capacity = math.ceil( len(valve_data) / num_batches )

    values = [ 1 for _ in range(len(valve_data))]
    weights = list(zip(valve_data['gpm_int'].tolist(), valve_data['Valve'].tolist()))
    weights.sort(key=lambda x: x[0])


    all_groups = []
    for run_ in tqdm(range(num_batches)):

        group_ = knapsack(weights, values, pump_unit_estimated_gpm, batch_capacity)
        all_groups.append(copy(group_))
        for element in group_:
            weights.remove(element)
    
    inverted_groups = []
    for idx, group_ in enumerate(all_groups):


        interm_group = []
        for x in group_:
            
            if isinstance(data.loc[data['Valve'] == x[-1]]['gpm'].item(), list):

                interm_group.append(( x[-1], data.loc[data['Valve'] == x[-1]]['gpm'].item() ))
            else:
                interm_group.append(( x[-1], data.loc[data['Valve'] == x[-1]]['gpm'].item() ))
        group_ = interm_group
        names = [ x[0] for x in group_]
        values = [ x[1] for x in group_]

        group_total_gpm = sum(values)
        number_of_pumps_required = math.ceil(group_total_gpm/per_pump_gpm)

        #inverted_groups.append((f"Group ID: {idx}", "", 
        #                        f"Group Total GPM: {group_total_gpm}",
        #                         f"Pump Works: {number_of_pumps_required}/{pump_type}",
        #                        f"Number of Valves: {len(names)}"))
        
        ##* Add groups:

        groups = { k.split("-")[0]: [] for k in names }

        for valve_name, valve_gpm in zip(names, values):
            key = valve_name.split("-")[0]
            groups[key].append((valve_name, valve_gpm))
        inverted_groups.append(groups)
        #inverted_groups.append(names)
        #inverted_groups.append(values)
    
    ##* map to controllers
    # check all groups with controllers
    # step 1: get all controlelrs

    
    #df = pd.DataFrame(inverted_groups).fillna("")

    return inverted_groups




In [20]:
data = pd.read_csv("../data/05-IRRIGATION MAINLINE  NETWORK.txt", header=0, sep='\t')
data = data.rename(columns ={
                    "AA": "Valve",
                    "20": "gpm"
                })
pump_unit_estimated_gpm = 1305


In [23]:
TWIN_PUMP = 2
TRIPLET_PUMP = 3

twin_pump_decimals = [ 0, 1, 2, 4, 5, 8, 9 ]
triplet_pump_decimals = [ 3, 7 ] 

data['gpm_int'] = data['gpm'].apply(lambda x: math.ceil(x))

data['valve_type_key'] = data.Valve.astype(str).apply(lambda x: x.strip().split('-')[-1][0])

valve_type_keys = data.valve_type_key.unique().tolist()

per_key_valves = data.loc[data['valve_type_key'] == 'B'][['Valve', 'gpm_int', 'gpm']]  
df= get_groups(per_key_valves)

 

100%|██████████| 6/6 [00:13<00:00,  2.27s/it]


In [27]:
column_keys = { k  for x in df for k in x.keys()  }
column_keys

{'A', 'B', 'C', 'D', 'E', 'F'}

In [33]:
controller_cell_spans = { k : max([ len(x[k]) if k in x else 0 for x in df ])   for k in column_keys }
controller_cell_spans

{'B': 7, 'D': 7, 'F': 3, 'E': 5, 'A': 8, 'C': 9}

In [43]:
headers =  "Group ID,"+ ", ".join(f"Controller {key}" + "," *(controller_cell_spans[key]-1)  for key in sorted(column_keys))
headers

'Group ID,Controller A,,,,,,,, Controller B,,,,,,, Controller C,,,,,,,,, Controller D,,,,,,, Controller E,,,,, Controller F,,'

In [54]:
dataframe_lines = [headers]
for idx, group in enumerate(df):
    names_line = f"## {idx}, "
    values_line = f", "
    for key in column_keys:
        
        group_vals = group.get(key, [])
        names_line += ", ".join([gv[0] for gv in group_vals])

        padding = ","*(controller_cell_spans[key]-len(group_vals))
        names_line += padding

        values_line += ", ".join([str(gv[-1]) for gv in group_vals])
        values_line += padding

    dataframe_lines.append(names_line)
    dataframe_lines.append(values_line)


dataframe_lines_ = [ x.split(",") for x in dataframe_lines ]

In [56]:
x = pd.DataFrame.from_records(dataframe_lines_)

In [62]:
max([ len(a) for a in dataframe_lines_])

40

In [61]:
x

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,Group ID,Controller A,,,,,,,,Controller B,...,,,Controller E,,,,,Controller F,,
1,## 0,B-B42,B-B74,B-B51,B-B65,B-B44,B-B72,D-B119,D-B120,D-B109,...,C-B95,C-B77,C-B75,,,None,None,None,None,None
2,,24.75,24.75,23.25,24.0,18.75,18.0,25.75,24.0,21.75,...,20.25,15.5,10.5,,,None,None,None,None,None
3,## 1,B-B57,B-B53,B-B43,B-B41,B-B59,B-B62,B-B52D-B117,D-B128,,...,,,,,,None,None,None,None,None
4,,39.0,36.0,34.5,32.25,32.25,30.75,29.2542.0,38.25,,...,,,,,,None,None,None,None,None
5,## 2,B-B71,B-B55,B-B58,B-B67,B-B64,B-B69,B-B54D-B112,D-B110,D-B129,...,C-B94,C-B82,C-B103,C-B99,C-B101,C-B90,None,None,None,None
6,,48.75,48.0,48.0,48.0,45.0,42.75,42.048.0,43.5,43.5,...,46.5,46.5,44.25,43.5,42.75,41.25,None,None,None,None
7,## 3,B-B56,B-B68,B-B70,B-B60,B-B47,B-B45,D-B116,D-B121,D-B130,...,C-B88,,,,,None,None,None,None,None
8,,57.75,56.25,57.0,53.25,52.5,49.5,58.0,57.75,54.5,...,50.25,,,,,None,None,None,None,None
9,## 4,B-B61,B-B63,B-B73,,,,D-B126,D-B127,D-B111,...,C-B106,,,,,,None,None,None,None


In [65]:
for i in x.columns:
    print(i)
    break

0


In [ ]:
gridOptions = {
    "columnDefs": [
        {
            'field': 0,
            'colSpan': "params => params.data.country === 'Russia' ? 2 : 1",
            'rowSpan': "params => params.data.country === 'Russia' ? 2 : 1",
        }
    ]
}

In [70]:
aa = [ list(a_.values()) for a_ in df]

In [75]:
aa

[[[('E-B137', 30.0), ('E-B142', 17.0)],
  [('C-B78', 28.5),
   ('C-B85', 28.5),
   ('C-B102', 26.25),
   ('C-B105', 27.0),
   ('C-B95', 20.25),
   ('C-B77', 15.5),
   ('C-B75', 10.5)],
  [('F-B152', 29.0), ('F-B149', 24.0), ('F-B150', 24.0)],
  [('A-B37', 26.25), ('A-B05', 22.0), ('A-B09', 21.0)],
  [('D-B119', 25.75),
   ('D-B120', 24.0),
   ('D-B109', 21.75),
   ('D-B123', 20.25),
   ('D-B113', 16.5),
   ('D-B108', 15.75)],
  [('B-B42', 24.75),
   ('B-B74', 24.75),
   ('B-B51', 23.25),
   ('B-B65', 24.0),
   ('B-B44', 18.75),
   ('B-B72', 18.0)]],
 [[('D-B117', 42.0), ('D-B128', 38.25)],
  [('E-B140', 42.0),
   ('E-B136', 37.0),
   ('E-B138', 35.0),
   ('E-B155', 34.0),
   ('E-B141', 31.0)],
  [('A-B31', 40.5),
   ('A-B03', 36.5),
   ('A-B21', 36.0),
   ('A-B29', 36.0),
   ('A-B22', 34.5),
   ('A-B16', 33.0),
   ('A-B32', 32.25)],
  [('F-B148', 40.5), ('F-B153', 32.0), ('F-B147', 32.0)],
  [('B-B57', 39.0),
   ('B-B53', 36.0),
   ('B-B43', 34.5),
   ('B-B41', 32.25),
   ('B-B59', 32.

In [78]:
from itertools import chain
list((chain.from_iterable(aa[0])))

[('E-B137', 30.0),
 ('E-B142', 17.0),
 ('C-B78', 28.5),
 ('C-B85', 28.5),
 ('C-B102', 26.25),
 ('C-B105', 27.0),
 ('C-B95', 20.25),
 ('C-B77', 15.5),
 ('C-B75', 10.5),
 ('F-B152', 29.0),
 ('F-B149', 24.0),
 ('F-B150', 24.0),
 ('A-B37', 26.25),
 ('A-B05', 22.0),
 ('A-B09', 21.0),
 ('D-B119', 25.75),
 ('D-B120', 24.0),
 ('D-B109', 21.75),
 ('D-B123', 20.25),
 ('D-B113', 16.5),
 ('D-B108', 15.75),
 ('B-B42', 24.75),
 ('B-B74', 24.75),
 ('B-B51', 23.25),
 ('B-B65', 24.0),
 ('B-B44', 18.75),
 ('B-B72', 18.0)]

In [79]:
[""]*4

['', '', '', '']

In [80]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/ag-grid/olympic-winners.csv"
)


In [84]:
df.loc[df.country == 'Russia']

,athlete,age,country,year,date,sport,gold,silver,bronze,total
4,Aleksey Nemov,24.0,Russia,2000,1/10/2000,Gymnastics,2,1,3,6
29,Aliya Mustafina,17.0,Russia,2012,12/8/2012,Gymnastics,1,1,2,4
31,Dmitry Sautin,26.0,Russia,2000,1/10/2000,Diving,1,1,2,4
120,Svetlana Khorkina,21.0,Russia,2000,1/10/2000,Gymnastics,1,2,0,3
121,Yekaterina Lobaznyuk,17.0,Russia,2000,1/10/2000,Gymnastics,0,2,1,3
...,...,...,...,...,...,...,...,...,...,...
8478,Yuliya Sotnikova,29.0,Russia,2000,1/10/2000,Athletics,0,0,1,1
8484,Maksim Tarasov,29.0,Russia,2000,1/10/2000,Athletics,0,0,1,1
8503,Yelena Yelesina,30.0,Russia,2000,1/10/2000,Athletics,1,0,0,1
8507,Olesya Zykina,19.0,Russia,2000,1/10/2000,Athletics,0,0,1,1


In [1]:
import streamlit_authenticator as stauth


In [2]:
hashed_passwords = stauth.Hasher(['typical@1234', 'def']).generate()


In [3]:
hashed_passwords

['$2b$12$zlBymTEr5eOmRrOjfeSUT.yiakYAcxbISTUvTLLTPLsWUOD9svv3S',
 '$2b$12$ssUEmFN3a0klxejooPmLuucuWrf7W5ls2GrWHDBJH6PSwzKlHljk.']

## Modifying the algorithm to reduce lowest batches

In [17]:
from model.time_scheduling import TimeSchedulingAlgorithm as tsa
from utils import Utils as ut
from constants import network_type_map
from itertools import chain


In [2]:
import pandas as pd

In [6]:
data = pd.read_csv("../data/New/NOTATION.txt", header=0, sep='\t')
data = data.rename(columns={"AA": "Valve", "20": "gpm"})
data.head(2)

,HANDLE,BLOCKNAME,Valve,2,gpm,GPM,WATERSOURCE/ZONEID
0,'4D8D,*U214,C-D426,"1.5""",26.9,GPM,يناير-23
1,'4DA3,*U214,C-D427,"1.5""",25.8,GPM,يناير-24


In [9]:
pump_type, pump_type_name = ut.get_pump_type(data, 1150)


In [106]:
valve_type_keys = data.valve_type_key.unique().tolist()

networks_batches = []

for network_key in ["B"]:

    
    per_key_valves = data.loc[data["valve_type_key"] == network_key][
        ["Valve", "gpm_int", "gpm"]
    ]

    solution_dictionary =  tsa.distibute_valves_into_batches(per_key_valves, 1150)
    

    column_keys = {k for x in solution_dictionary for k in x.keys()}

    controller_cell_spans = {
        k: max([len(x[k]) if k in x else 0 for x in solution_dictionary])
        for k in column_keys
    }

    headers = "Batch ID, # Valves / Batch, Total Batch GPM, " + ", ".join(
        f"Controller {key}" + "," * (controller_cell_spans[key] - 1)
        for key in sorted(column_keys)
    )
    dataframe_lines = [headers]

    for idx, group in enumerate(solution_dictionary):

        flattened = list((chain.from_iterable(group.values())))

        total_gpm = "{:5.3f}".format(sum([f_[-1] for f_ in flattened]))
        names_line = f"Batch #{idx+1}, {len(flattened)}, { total_gpm }, "
        values_line = f", , , "
        for key in sorted(column_keys):

            group_vals = group.get(key, [])
            names_line += ", ".join([gv[0] for gv in group_vals])

            padding = "," * (
                controller_cell_spans[key] - len(group_vals) + 1
            )
            names_line += padding

            values_line += ", ".join(
                ["{:5.3f}".format(gv[-1]) for gv in group_vals]
            )
            values_line += padding
            ##* correction step:
            if values_line.strip()[-1] != ",":
                values_line += ", "
            if names_line.strip()[-1] != ",":
                names_line += ", "
        dataframe_lines.append(names_line)
        dataframe_lines.append(values_line)

    dataframe_lines_ = [x.split(",") for x in dataframe_lines]

    max_width = max([len(x_) for x_ in dataframe_lines_])

    dataframe_lines_.append([] * max_width)
    
    network_schedule = pd.DataFrame.from_records(
        dataframe_lines_,
        columns=[ut.num_to_col(i_ + 1) for i_ in range(max_width)],
    )

    header = f'{ network_type_map.get(network_key, network_key)} Network,    Total GPM = {  "{:.3f}".format(per_key_valves["gpm"].sum())},\tNumber of valves = {len(per_key_valves["gpm"])}'


    networks_batches.append((key, controller_cell_spans, header, network_schedule))
    #break

100%|██████████| 162/162 [00:00<00:00, 1709.94it/s]


In [21]:
best_per_pump_gpm = 1150 /pump_type
best_per_pump_gpm

575.0

In [100]:
networks_batches[0][-1]

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,AA,AB,AC,AD,AE,AF,AG,AH,AI,AJ,AK,AL,AM,AN,AO,AP,AQ,AR,AS,AT,AU,AV,AW,AX,AY,AZ,BA,BB,BC,BD,BE,BF,BG,BH,BI,BJ,BK,BL,BM,BN,BO,BP,BQ,BR,BS,BT,BU,BV,BW,BX,BY,BZ,CA
0,Batch ID,# Valves / Batch,Total Batch GPM,Controller A,,,,,,,,,,,,,,,,,,,,Controller B,,,,,,,,,,,,,Controller C,,,,,,,,,,,,,,,,,,Controller D,,,,,,,,,Controller E,,,,,,,,,,,,,,None,None
1,Batch #1,70,1117.250,A-B80,A-B78,A-B69,A-B58,A-B155,A-B73,A-B71,A-B152,A-B87,A-B154,A-B68,A-B59,A-B57,A-B54,A-B61,A-B60,A-B56,A-B74,A-B51,A-B50,B-B117,B-B158,B-B147,B-B114,B-B145,B-B115,B-B79,B-B118,B-B116,,,,,C-B137,C-B106,C-B102,C-B131,C-B97,C-B136,C-B134,C-B112,C-B108,C-B107,C-B159,C-B110,C-B109,C-B133,C-B103,C-B100,C-B101,C-B132,D-B34,D-B28,D-B26,D-B157,D-B35,D-B29,D-B27,D-B161,D-B162,E-B08,E-B07,E-B05,E-B21,E-B16,E-B13,E-B12,E-B01,E-B49,E-B22,E-B04,E-B03,E-B15,E-B06,,None
2,,,,20.000,19.500,19.500,18.750,17.250,17.250,18.000,16.500,16.500,15.750,14.250,14.250,15.000,14.250,13.500,12.750,12.000,10.500,9.750,9.000,18.750,17.250,17.250,18.000,15.000,15.000,15.000,13.500,12.750,,,,,19.500,19.500,19.500,18.750,18.750,17.250,18.000,18.000,18.000,17.250,16.500,16.500,15.750,14.250,15.000,14.250,10.500,6.750,19.500,19.500,17.250,16.500,16.500,14.250,14.250,10.000,8.750,19.500,19.500,19.500,18.750,18.500,19.000,18.750,18.750,16.500,16.500,16.500,15.750,12.750,9.750,,None
3,Batch #2,50,1107.500,A-B62,A-B153,A-B86,A-B85,A-B65,A-B84,A-B82,A-B81,A-B53,,,,,,,,,,,,B-B140,B-B123,B-B91,B-B120,B-B19,B-B119,B-B148,B-B146,B-B142,B-B141,B-B138,B-B149,B-B113,C-B90,C-B111,C-B99,C-B105,C-B135,C-B104,C-B93,,,,,,,,,,,,D-B37,D-B36,D-B41,D-B23,D-B38,D-B25,D-B39,D-B24,,E-B45,E-B42,E-B48,E-B46,E-B18,E-B14,E-B160,E-B156,E-B47,E-B44,E-B43,E-B10,E-B02,,,None
4,,,,25.250,24.750,24.750,23.250,23.250,22.500,20.250,20.250,21.000,,,,,,,,,,,,24.750,24.750,25.000,23.250,23.250,22.500,20.250,21.000,21.000,20.250,21.000,19.500,19.500,25.000,24.000,22.500,21.750,21.000,20.250,20.250,,,,,,,,,,,,25.500,24.750,24.000,23.250,22.500,21.750,21.000,21.000,,24.000,24.000,21.750,21.750,21.250,21.750,21.000,20.250,20.250,20.250,21.000,20.250,20.250,,,None
5,Batch #3,34,1119.750,A-B76,A-B63,A-B72,A-B66,A-B70,A-B67,A-B89,A-B55,A-B83,A-B95,,,,,,,,,,,B-B130,B-B122,B-B151,B-B124,B-B126,B-B125,B-B150,B-B139,B-B128,B-B144,B-B143,B-B121,,C-B92,C-B98,C-B96,C-B77,C-B94,,,,,,,,,,,,,,D-B33,D-B31,D-B32,D-B40,D-B30,,,,,E-B17,E-B09,,,,,,,,,,,,,,None
6,,,,40.000,36.250,35.500,33.000,31.500,30.250,29.000,27.750,27.000,25.500,,,,,,,,,,,37.750,37.500,33.750,33.750,32.250,32.250,31.500,31.500,31.750,29.250,28.500,25.500,,39.000,37.500,37.250,37.250,35.250,,,,,,,,,,,,,,40.750,32.250,31.000,27.000,27.000,,,,,39.500,35.000,,,,,,,,,,,,,,None
7,Batch #4,8,369.250,A-B64,A-B88,A-B75,,,,,,,,,,,,,,,,,,B-B127,B-B129,,,,,,,,,,,,C-B52,,,,,,,,,,,,,,,,,,,,,,,,,,,,E-B20,E-B11,,,,,,,,,,,,,
8,,,,53.000,46.250,42.000,,,,,,,,,,,,,,,,,,47.750,45.500,,,,,,,,,,,,43.500,,,,,,,,,,,,,,,,,,,,,,,,,,,,46.000,45.250,,,,,,,,,,,,,
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [39]:
def within_any_range(batch_total_gpm, pump_gpm, pump_type_):

    ##* acceptable_ranges:

    nearest_gpm, nearest_gpm_delta = -1, 99999

    pump_motor_gpm = pump_gpm/pump_type_
    #accepted_ranges = [ (pump_motor_gpm*i*0.9,pump_motor_gpm*i, pump_motor_gpm*i*1.1)  for i in range(1, pump_type+1) ] 

    for range_indx in range(1, pump_type_+1):
        accepted_range = pump_motor_gpm*range_indx
        delta = abs(accepted_range-batch_total_gpm)
        if delta <= nearest_gpm_delta:
            nearest_gpm_delta = delta
            nearest_gpm = accepted_range
            
        if delta<=0.1*accepted_range:
            return True, nearest_gpm
    #* if reached here, then no acceptable range, return nearest one:
    return False, nearest_gpm

In [48]:
def get_group_total_gpm(group):
    flattened = list((chain.from_iterable(group.values())))
    return sum([f_[-1] for f_ in flattened]), flattened

In [105]:
def correct_invalid_group(incorrect_group, remaining_groups, optimal_gpm, pump_gpm, pump_type):

    total_gpm, flattened = get_group_total_gpm(incorrect_group)
    ## Keep removing valves until the batch is corrected 
    found_correction = False
    while total_gpm - optimal_gpm > optimal_gpm*0.1:
        #* first, choose the item to be replaced: the one that minimizes difference:
        delta_gp = abs(optimal_gpm-total_gpm)
        #sorted_valves_by_delta_gpm = sorted(flattened, key=lambda x: abs(x[-1]-delta_gp))
        sorted_valves_by_delta_gpm = sorted(flattened, key=lambda x: x[-1])
        valve_to_replace = sorted_valves_by_delta_gpm[0]        
        ##* choose the batch from the remaining groups that optimizes the performance
        remaining_groups.sort(key=lambda x: get_group_total_gpm(x)[0])
        found_correction = False
        for group_ in remaining_groups:
            current_group_gpm = get_group_total_gpm(group_)[0]
            if  within_any_range(current_group_gpm + valve_to_replace[1],pump_gpm, pump_type)[0]:
                valve_key = valve_to_replace[0].split("-")[0]
                group_[valve_key].append(valve_to_replace)
                incorrect_group[valve_key].remove(valve_to_replace)
                del valve_to_replace
                found_correction = True
                break
            
        if not found_correction:
            print("No possible corrections for this solution")
            break
        else:
            print("made correction to the solution for optimality")
        total_gpm, flattened = get_group_total_gpm(incorrect_group)

    remaining_groups.append(incorrect_group)
    return remaining_groups, found_correction


In [107]:
has_correction = True

while has_correction:

    for idx in range(len(solution_dictionary)):
        group = solution_dictionary[idx]
        has_correction = False

        flattened = list((chain.from_iterable(group.values())))

        total_gpm = "{:5.3f}".format(sum([f_[-1] for f_ in flattened]))

        check_flag, nearest_gpm = within_any_range(float(total_gpm), 1150, pump_type)
        if check_flag:
            print(f"Batch #{idx} with GPM = {total_gpm} is valid with nearest GPM = {nearest_gpm}")
        else:
            print(f"Batch #{idx} with GPM = {total_gpm} is NOT valid with nearest GPM = {nearest_gpm}")
            #has_correction = True
            solution_dictionary.remove(group)
            solution_dictionary, did_correction = correct_invalid_group(group, solution_dictionary, nearest_gpm, 1150, pump_type)
            has_correction = did_correction



Batch #0 with GPM = 1117.250 is valid with nearest GPM = 1150.0
Batch #1 with GPM = 1107.500 is valid with nearest GPM = 1150.0
Batch #2 with GPM = 1119.750 is valid with nearest GPM = 1150.0
Batch #3 with GPM = 369.250 is NOT valid with nearest GPM = 575.0


In [56]:
has_correction=True
while has_correction:

    for idx in range(len(solution_dictionary)):
        group = solution_dictionary[idx]
        has_correction = False

        flattened = list((chain.from_iterable(group.values())))

        total_gpm = "{:5.3f}".format(sum([f_[-1] for f_ in flattened]))

        check_flag, nearest_gpm = within_any_range(float(total_gpm), 1150, pump_type)
        if check_flag:
            print(f"Batch #{idx} with GPM = {total_gpm} is valid with nearest GPM = {nearest_gpm}")
        else:
            #has_correction = True
            #solution_dictionary.remove(group)
            #solution_dictionary = correct_invalid_group(group, solution_dictionary, nearest_gpm, 1150, pump_type)
            print(f"Batch #{idx} with GPM = {total_gpm} is NOT valid with nearest GPM = {nearest_gpm}")



Batch #0 with GPM = 1102.090 is valid with nearest GPM = 1150.0
Batch #1 with GPM = 1107.500 is valid with nearest GPM = 1150.0
Batch #2 with GPM = 1135.280 is valid with nearest GPM = 1150.0
Batch #3 with GPM = 1114.350 is valid with nearest GPM = 1150.0
Batch #4 with GPM = 1133.300 is valid with nearest GPM = 1150.0
Batch #5 with GPM = 1135.740 is valid with nearest GPM = 1150.0
Batch #6 with GPM = 1135.220 is valid with nearest GPM = 1150.0
Batch #7 with GPM = 1107.820 is valid with nearest GPM = 1150.0
Batch #8 with GPM = 1097.750 is valid with nearest GPM = 1150.0
Batch #9 with GPM = 1108.400 is valid with nearest GPM = 1150.0
Batch #10 with GPM = 1096.800 is valid with nearest GPM = 1150.0
Batch #11 with GPM = 1111.180 is valid with nearest GPM = 1150.0
Batch #12 with GPM = 1098.100 is valid with nearest GPM = 1150.0
Batch #13 with GPM = 649.620 is NOT valid with nearest GPM = 575.0


In [108]:
import streamlit_authenticator as stauth


In [110]:
stauth.Hasher(["12345678"]).generate()

['$2b$12$CBhOgt70GZsv1Flw/QoRuuwTh7N4vtw3BfMGW3lK3Dybmbn85tsFK']

In [2]:
data = pd.read_csv("./projects/Neom/Bubbler Network.csv")

In [3]:
data.head(2)

,A,B,C,D,E,F,G,H,I,J,...,AY,AZ,BA,BB,BC,BD,BE,BF,BG,BH
0,Batch ID,# Valves / Batch,Total Batch GPM,Controller E,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Batch #1,47,1283.500,E-B84,E-B86,E-B110,E-B76,E-B64,E-B109,E-B60,...,K-B160,K-B144,K-B155,K-B134,NaN,NaN,NaN,NaN,NaN,NaN
